In [ ]:
import numpy as np
import random
from sklearn.cluster import KMeans
#import pandas as pd
#import re
import itertools
#from numpy import random

#### Code permettant de déterminer le score de similarité entre deux modèles à bloc stochastique :

In [ ]:
def permut(pi):
    a = len(pi)
    b = np.math.factorial(a)
    l = list(itertools.permutations(pi,a))
    l_per = np.zeros((b,a))
    k = 0
    for i in l:
        i = np.array(i)
        l_per[k] = i
        k = k+1
    return l_per

def cop(i,j,gamma,gamma2):
    for k in range(len(gamma[0])):
        gamma[j][k] = gamma2[i][k].copy()
    return gamma


def inv_per(teta,pi_perm):
    a = len(pi_perm)
    gamma = teta[1]
    gamma_inv = np.zeros((a,a))
    for i in range (a):
        for j in range (a):
            if (teta[0][i] == pi_perm[j]):
                for k in range(a):
                    gamma_inv[k][j] = gamma[k][i]
    gamma_inv2 = gamma_inv.copy()
    for i in range (a):
        for j in range (a):
            if (teta[0][i] == pi_perm[j]):
                gamma_inv = cop(i,j,gamma_inv,gamma_inv2)
            #j = 0
            #while((teta[0][i] != pi_perm[j]) & (j < a-1)):
                #j = j+1
            #gamma_inv = inv_l(i,j,gamma_inv,gamma_inv2)
        
    teta_inv = [pi_perm,gamma_inv]
    return teta_inv


def Calcul_sim(teta1,teta2):
    pi1 = teta1[0]
    gamma1 = teta1[1]
    teta_2 = teta2
    pi2 = teta_2[0]
    gamma2 = teta_2[1]
    err_pi = 0
    err_gamma = 0
    a = len(pi1)
    for i in range(a):
        if(pi1[i] != 0):
            err_pi += 0.5*np.abs((pi1[i] - pi2[i])/pi1[i])
        if(pi2[i] != 0):
            err_pi += 0.5*np.abs((pi1[i] - pi2[i])/pi2[i])
        for j in range(a):
            if (gamma1[i][j] != 0):
                err_gamma += 0.5*np.abs((gamma1[i][j] - gamma2[i][j])/gamma1[i][j])
            if (gamma2[i][j] != 0):
                err_gamma += 0.5*np.abs((gamma1[i][j] - gamma2[i][j])/gamma2[i][j])
    sim = np.exp(-(err_pi + err_gamma))
    return sim

def Meilleur_sim(tet1,tet2):
    sim = Calcul_sim(tet1,tet2)
    a = len(tet1[0])
    i = 0
    k = np.math.factorial(a)
    pi1_Q = np.arange(a)
    pi1_M = tet1[0].copy()
    pi2_Q = conv_r_Q(tet2[0],pi1_M,pi1_Q)
    gamma1_Q = tet1[1].copy()
    gamma2_Q = tet2[1].copy()
    teta1 = [pi1_Q,gamma1_Q]
    teta2 = [pi2_Q,gamma2_Q]
    l_permut = np.zeros((k,a))
    l_permut = permut(pi2_Q)
    while(i != np.math.factorial(a)):
        teta2_inv = inv_per(teta2,l_permut[i])
        sim_inv = Calcul_sim(teta1,teta2_inv)
        if (sim_inv > sim):
            sim = sim_inv
        i = i + 1
    if (sim > sim_inv):
        return sim
    return sim_inv

def conv_r_Q(pi_r,piM,piQ):
    pi = pi_r.copy()
    for i in range (len(piQ)):
        for j in range (len(piQ)):
            if (pi[i] == piM[j])&((piQ[j] in pi)==False):
                pi[i] = piQ[j]
    return pi

#### Code Générant une matrice d'adjacence selon un modèle à bloc stochastique :

In [ ]:
random.seed()
def SBM(Pi, Gamma, Q, n):
    Adjacence = np.zeros((n,n))
    classes = np.arange(1,Q+1)
    classes_noeuds = np.zeros(n)
    
    for i in range(n):#On initialise les classes de tous les noeuds
        val = np.random.randint(1000)/1000
        n_class = random.choices(classes, weights = Pi, k=1)[0]
        classes_noeuds[i] = n_class
    
    
    for i in range(n):
        for j in range(n):
            if i != j:
                val = np.random.randint(1000)/1000
                if val<=Gamma[int(classes_noeuds[i]-1)][int(classes_noeuds[j]-1)] :
                    Adjacence[i,j] = 1
                else:
                    Adjacence[i,j] = 0
            else:
                Adjacence[i,j] = 0
    return Adjacence, classes_noeuds

#### Initialisation de l'algorithme + Algorithme VEM :

In [ ]:
def init(M,Q):
    kmeans = KMeans(n_clusters=Q).fit_predict(M)
    Tau = np.zeros((M.shape[0],Q))
    for i in range(M.shape[0]):
        for j in range(Q):
            if kmeans[i] == j:
                Tau[i,j] = 1
            else:
                Tau[i,j] = 0
    return Tau

In [ ]:
def VEM(M,Q,T):
    #Initialisation
    Tau = init(M,Q)
    Pi = np.zeros(Q)
    Gamma = np.zeros((Q,Q))
    n = M.shape[0]
    #Étape A
    for i in range(T):
        for q in range(Q):
            Pi[q] = 1/n * sum(Tau[:,q])
        for q in range(Q):
            for l in range(Q):
                Gamma[q,l] = inner_sum(Tau,M,q,l)
    #Étape B
    for k in range(10):
        for i in range(n):
            for q in range(Q):
                Tau[i,q] = inner_prod(Gamma,Pi,Tau,M,Q,q,i)
            somme = sum(Tau[i,:])
            for q in range(Q):
                if somme != 0:
                    Tau[i,q] = Tau[i,q]/somme
                else:
                    Tau[i,q] = 0
    return Pi,Gamma,Tau
            
            
            
            
def inner_sum(Tau,M,q,l):
    Num = 0
    Den = 0
    for i in range(Tau.shape[0]):
        for j in range(Tau.shape[0]):
            if i!=j:
                Num+=Tau[i,q]*Tau[j,l]*M[i,j]
                Den+=Tau[i,q]*Tau[j,l]
    if Den == 0:
        return 0
    return Num/Den

def inner_prod(Gamma,Pi,Tau,M,Q,q,i):
    prod = 1
    for j in range(M.shape[0]):
        for l in range(Q):
            prod *= pow(pow(Gamma[q,l],M[i,j])*pow(1-Gamma[q,l],1-M[i,j]),Tau[j,l])
    return Pi[q]*prod

#### Critère ICL :

In [ ]:
SMALL = 1e-10
def ICL(M,Q,n):
    Pi,Gamma,Tau = VEM(M,Q,100)
    sum1 = 0
    sum2 = 0
    for i in range(n):
        for q in range(Q):
            #print(i,q)
            sum1 += Tau[i,q]*np.log(Pi[q])
    
    for i in range(n):
        for j in range(n):
            if i != j:
                for q in range(Q):
                    for l in range(Q):
                        #print(i,j,q,l)
                        mul = Tau[i,q]*Tau[j,l]
                        #print("mul :",mul)
                        #print("M[i,j] :",M[i,j])
                        #print(mul*(M[i,j]*np.log(Gamma[q,l]) + (1-M[i,j])*np.log(1-Gamma[q,l])))
                        #sum2 += mul*(M[i,j]*np.log(Gamma[q,l]) + (1-M[i,j])*np.log(1-Gamma[q,l]))
                        sum2 += mul*(M[i,j]*np.log(Gamma[q,l] + SMALL) + (1-M[i,j])*np.log(1-Gamma[q,l] + SMALL))
    A = sum1 + sum2
    icl = A - 0.5*(Q-1)*np.log(n) - 0.5*Q*(Q+1)*0.5*np.log(n*(n-1)/2)
    return icl

In [ ]:
import matplotlib.pyplot as plt

#### Création de la suite de toutes les matrices d'adjacence des graphes du Dataset Tox_21_ARE :

In [ ]:
%%time
def get_index(n,Lines,graph,start):
    count = -1
    for i in range(start,len(Lines)):
        #print(int(Lines[i]),graph)
        if int(Lines[i]) == graph:
            count+=1
            #print(i+1,Lines[i],count)
        if i+1 == n:
            return count

def create_df(filename1,filename2,filename3,matrix_len):
    file1 = open(filename1,'r')#liaisons
    file2 = open(filename2,'r')#quel graphe
    file3 = open(filename3,'r')#quelle classe
    Lines1 = file1.readlines()
    Lines2 = file2.readlines()
    Lines3 = file3.readlines()
    matrixes = [0 for i in range(matrix_len)]
    start = [0 for i in range(matrix_len)]
    sizes = np.zeros(matrix_len)
    classes = []
    count = -1
    count2 = 0
    val = 0
    length = 0
    #print(len(Lines2))
    for i in range(len(Lines2)):
        if int(Lines2[i]) != val:
            val = int(Lines2[i])
            start[count2] = i
            count2+=1
            sizes[count]+=1
            count+=1
        else:
            sizes[count]+=1
    
    for i in range(len(sizes)):
        matrixes[i] = np.zeros((int(sizes[i]),int(sizes[i])))
    
    for i in range(len(Lines3)):
        classes.append(int(Lines3[i]))
    
    compteur = 0
    for line in Lines1:
        #print(compteur)
        n1 = ""
        n2 = ""
        #print(line[0])
        i = 0
        while(line[i] != ','):
            n1+=line[i]
            i+=1
        i = i+1
        while(line[i]!='\n'):
            n2+=line[i]
            i+=1
        n1 = int(n1)
        n2 = int(n2)
        i = int(Lines2[n1-1])-1
        #print(n1,start[i])
        index1 = get_index(n1,Lines2,i+1,start[i])
        index2 = get_index(n2,Lines2,i+1,start[i])
        matrixes[i][index1,index2] = 1
        compteur+=1
        
    return matrixes,classes,sizes

matrixes,classes,sizes = create_df("Tox21_ARE_evaluation/Tox21_ARE_evaluation_A.txt","Tox21_ARE_evaluation/Tox21_ARE_evaluation_graph_indicator.txt","Tox21_ARE_evaluation/Tox21_ARE_evaluation_graph_labels.txt",552)        
matrixesT,classesT,sizesT = create_df("Tox21_ARE_testing/Tox21_ARE_testing_A.txt","Tox21_ARE_testing/Tox21_ARE_testing_graph_indicator.txt","Tox21_ARE_testing/Tox21_ARE_testing_graph_labels.txt",234)
matrixesG,classesG,sizesG = create_df("Tox21_ARE_training/Tox21_ARE_training_A.txt","Tox21_ARE_training/Tox21_ARE_training_graph_indicator.txt","Tox21_ARE_training/Tox21_ARE_training_graph_labels.txt",7167)


# Détermination du nombre optimal de classes pour l'ensemble de graphes

#### On teste le critère ICL sur 10 graphes différents pris à des endroits différents du Dataset pour un nombre de classes de noeuds variant entre 2 et 5 puis on garde le nombre optimal de classes du Dataset comme étant la médiane du nombre optimal de classes pour chacun des graphes testés 

In [ ]:
import operator

In [ ]:
%%time
sample = 10
elements = np.zeros(sample)

for i in range(sample):
    #print(i)
    N = np.arange(2,6)
    #print(N)
    icls = [ICL(matrixes[i*13],k,matrixes[i*13].shape[0]) for k in range(2, 6)]
    #print(icls)
    #plt.plot(N,np.array(icls))
    #plt.show()
    index, value = max(enumerate(icls), key=operator.itemgetter(1))
    elements[i] = index+2
    
    

In [ ]:
print(elements)

Q = int(np.median(elements))
print("Le nombre optimal de classes retenu est de ",Q)
Q = 2

#### Pour éviter les cas où un graphe possède moins de sommets que de nombre de classes de noeuds différentes dans son modèle à bloc stochastique, on retire du jeu de données d'entraînement les graphes possédant cette caractéristique

In [ ]:
matrixes2 = []
classes2 = []
sizes2 = []
indexes = []
for i in range(len(matrixes)):
    if matrixes[i].shape[0]>=Q:
        matrixes2.append(matrixes[i])
        classes2.append(classes[i])
        sizes2.append(sizes[i])
    else:
        indexes.append(i)
matrixes = matrixes2.copy()
classes = classes2.copy()

#### On effectue la même opération sur le jeu de données de test de taille moyenne

In [ ]:
matrixes2T = []
classes2T = []
sizes2T = []
indexesT = []
for i in range(len(matrixesT)):
    if matrixesT[i].shape[0]>=Q:
        matrixes2T.append(matrixes[i])
        classes2T.append(classes[i])
        sizes2T.append(sizes[i])
    else:
        indexesT.append(i)
classesT = classes2T.copy()
matrixesT = matrixes2T.copy()

#### On effectue la même opération sur le jeu de données de test de grande taille

In [ ]:
matrixes2G = []
classes2G = []
sizes2G = []
indexesG = []
for i in range(len(matrixesG)):
    if matrixesG[i].shape[0]>=Q:
        matrixes2G.append(matrixesG[i])
        classes2G.append(classesG[i])
        sizes2G.append(sizesG[i])
    else:
        indexesG.append(i)
classesG = classes2G.copy()
matrixesG = matrixes2G.copy()

# Création du DataFrame

#### Grâce à l'algorithme VEM, on va déterminer une approximation des paramètres du modèle à bloc stochastique pour chaque graphe du jeu de données d'entraînement dans une liste :

In [ ]:
%%time
Infos = [0 for i in range(len(matrixes))]
for i in range(len(matrixes)):
    #print(i)
    Infos[i] = VEM(matrixes[i],Q,40)

#### On effectue le même procédé sur le jeu de données de test de taille moyenne :

In [ ]:
%%time
InfosT = [0 for i in range(len(matrixesT))]
for i in range(len(matrixesT)):
    #print(i)
    InfosT[i] = VEM(matrixesT[i],Q,40)

#### On effectue le même procédé sur le jeu de données d'entraînement de grande taille :

In [ ]:
%%time
InfosG = [0 for i in range(len(matrixesG))]
for i in range(len(matrixesG)):
    InfosG[i] = VEM(matrixesG[i],Q,10)

#### Création de la matrice de similarité entre les graphes de l'échantillon d'entraînement :

In [ ]:
%%time
X_train = np.zeros((len(Infos),len(Infos)))
len_train = len(Infos)
for i in range(len_train):
    for j in range(len_train):
        X_train[i,j] = Meilleur_sim(Infos[i],Infos[j])

#### Création de la matrice de similarité entre chaque graphes du jeu de données de test de taille moyenne et chaque graphe du jeu de données d'entraînement :

In [ ]:
%%time
len_test = len(matrixesT)
X_T = np.zeros((len_test,len(Infos)))
for i in range(len_test):
    for j in range(len(Infos)):
        X_T[i,j] = Meilleur_sim(InfosT[i],Infos[j])

#### Création de la matrice de similarité entre chaque graphe du jeu de données de test de grande taille et chaque graphe du jeu de données d'entraînement :

In [ ]:
%%time
X_G = np.zeros((len(InfosG),len(Infos)))
len_G = len(InfosG)
for i in range(len_G):
    for j in range(len_train):
        X_G[i,j] = Meilleur_sim(InfosG[i],Infos[j])

# Classification des différents graphes

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

#### Initialisation et entraînement de l'algorithme de machine learning sur le jeu de données d'entraînement :

In [ ]:
svc = SVC(kernel='precomputed')
svc.fit(X_train,classes)

### Prédictions de l'algorithme entraîné sur les jeux de données de test :

In [ ]:
y_predict = svc.predict(X_T)

In [ ]:
y_predict

In [ ]:
np.array(classesT)

#### Précision de l'algorithme sur le jeu de données de test de taille moyenne puis sur celui de grande taille :

In [ ]:
svc.score(X_T,classesT)

In [ ]:
svc.score(X_G,classesG)

In [ ]:
from joblib import dump, load
#dump(svc, 'svc_tox_precomputed_2.joblib')